In [1]:
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize
import folium
from haversine import haversine, Unit
import numpy as np

##### url 설정하기

In [2]:
url = 'https://www.starbucks.co.kr/store/getStore.do?r=N22B447K1D'
data = {
    'ins_lat' : '37.56682',
    'ins_lng' : '126.97865',
    'p_sido_cd' : '05',
    'in_biz_cd' : '',
    'set_date' : '',
    'iend' : '1000'}
response = requests.post(url,data = data )
busan_stores = response.json()
busan_stores

{'list': [{'seq': 0,
   'sido_cd': None,
   'sido_nm': None,
   'gugun_cd': None,
   'gugun_nm': None,
   'code_order': None,
   'view_yn': None,
   'store_num': None,
   'sido': None,
   'gugun': None,
   'address': None,
   'new_img_nm': None,
   'p_pro_seq': 0,
   'p_view_yn': None,
   'p_sido_cd': '',
   'p_gugun_cd': '',
   'p_store_nm': None,
   'p_theme_cd': None,
   'p_wireless_yn': None,
   'p_smoking_yn': None,
   'p_book_yn': None,
   'p_music_yn': None,
   'p_terrace_yn': None,
   'p_table_yn': None,
   'p_takeout_yn': None,
   'p_parking_yn': None,
   'p_dollar_assent': None,
   'p_card_recharge': None,
   'p_subway_yn': None,
   'stb_store_file_renew': None,
   'stb_store_theme_renew': None,
   'stb_store_time_renew': None,
   'stb_store_lsm': None,
   's_code': '1605',
   's_name': '금정남산동DT',
   'tel': '1522-3232',
   'fax': '051-582-3706',
   'sido_code': '05',
   'sido_name': '부산',
   'gugun_code': '0501',
   'gugun_name': '금정구',
   'addr': '부산광역시 금정구 남산동 68-1 남산프라자',


In [3]:
# df = json_normalize(busan_stores, 'list')
# df = df[['s_name', 'lat', 'lot', 'sido_name', 'gugun_name', 'doro_address', 'tel']]
# df[df['s_name'].str.contains('신세계')]

##### 데이터프레임 만들어보기

In [4]:
# DT 매장 위치
df = json_normalize(busan_stores, 'list')
df_dt = df[['s_name', 'lat', 'lot', 'sido_name', 'gugun_name', 'doro_address', 'tel']]
df_dt = df_dt[df_dt['s_name'].str.contains('DT')].reset_index(drop = True)
df_dt

C:\Users\juku\AppData\Local\Temp\ipykernel_5156\2769043734.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(busan_stores, 'list')


,s_name,lat,lot,sido_name,gugun_name,doro_address,tel
0,금정남산동DT,35.262466,129.091353,부산,금정구,부산광역시 금정구 중앙대로 1989 (남산동),1522-3232
1,금정구청DT,35.238784,129.092964,부산,금정구,부산광역시 금정구 중앙대로 1730 (부곡동),1522-3232
2,온천장역DT,35.224006,129.08656,부산,금정구,부산광역시 금정구 식물원로 13 (장전동),1522-3232
3,동명대DT,35.12311959047579,129.09901642703608,부산,남구,부산광역시 남구 신선로 423 (용당동),1522-3232
4,화명강변DT,35.216972,129.004611,부산,북구,부산광역시 북구 금곡대로77 (덕천동),1522-3232
5,부산구남역DT,35.1982412867577,128.996280148597,부산,북구,부산광역시 북구 백양대로 1040 (구포동),1522-3232
6,부산만덕DT,35.213166,129.039764,부산,북구,부산광역시 북구 만덕대로 323 (만덕동),1522-3232
7,사상주례DT,35.154847,128.996302,부산,사상구,부산광역시 사상구 백양대로 520 (주례동),1522-3232
8,동서대DT,35.150863,129.010026,부산,사상구,부산광역시 사상구 가야대로 338 (주례동),1522-3232
9,삼락강변DT,35.17654467870461,128.9766669978161,부산,사상구,부산광역시 사상구 낙동대로 1306 (삼락동),1522-3232


#### lat과 lot이 object 타입임을 확인하고, 기술통계를 통해 평균을 구하기 위해 int 로 바꾸어 준다.

In [5]:
df_dt.dtypes

s_name          object
lat             object
lot             object
sido_name       object
gugun_name      object
doro_address    object
tel             object
dtype: object

In [6]:
print(df_dt.dtypes)
df_dt = df_dt.astype({'lat':'float'})
df_dt = df_dt.astype({'lot':'float'})
print(df_dt.dtypes)

s_name          object
lat             object
lot             object
sido_name       object
gugun_name      object
doro_address    object
tel             object
dtype: object
s_name           object
lat             float64
lot             float64
sido_name        object
gugun_name       object
doro_address     object
tel              object
dtype: object


##### 위도, 경도의 평균 구하기

In [7]:
print(df_dt[['lat', 'lot']].describe(exclude = 'O').iloc[1,0])
print(df_dt[['lat', 'lot']].describe(exclude = 'O').iloc[1,1])

35.17516436765402
129.06573399335286


##### 모든 지점의 평균 위치를 중점으로 하여 DT매장들의 위치를 확인 해보자!

In [10]:
m = folium.Map(location=[35.171866, 129.059350], 
               zoom_start= 11)
for mark_lat, mark_lot in zip(df_dt['lat'], df_dt['lot']):
    folium.Marker([mark_lat, mark_lot]).add_to(m)
display(m)

#### 지역을 입력받고 지역에 해당하는 DT 매장의 목록 및 위치를 확인하는 함수를 만들어보자!

In [23]:
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize
import folium

In [24]:
def get_dt_location(location_name):
    '''
    지역명을 입력받아 지역에 해당하는 DT매장을 지도에 표시해주는 함수
    검색 가능 지역 : 전체, 서울, 경기, 광주, 대구, 대전, 부산, 울산,인천,
                     강원,경남, 경북, 전남, 전북, 충남, 충북, 제주, 세종
    '''
    try:
        # 검색 가능한 지역 목록
        location_dic = {
            '전체' : '00', '서울' : '01', '광주' : '02', '대구' : '03',
            '대전' : '04', '부산' : '05', '울산' : '06', '인천' : '07',
            '경기' : '08', '강원' : '09', '경남' : '10', '경북' : '11',
            '전남' : '12', '전북' : '13', '충남' : '14', '충북' : '15',
            '제주' : '16', '세종' : '17'
            }
        # 지역명을 입력받아 데이터를 불러오기 위한 번호 출력
        search_loc = location_dic[location_name]
        # 사이트에서 json으로 이루어진 정보 가져오기
        url = 'https://www.starbucks.co.kr/store/getStore.do?r=N22B447K1D'
        data = {
        'ins_lat' : '37.56682',
        'ins_lng' : '126.97865',
        'p_sido_cd' : search_loc,
        'in_biz_cd' : '',
        'set_date' : '',
        'iend' : '1000'}
        response = requests.post(url,data = data )
        busan_stores = response.json()
        # json_normal을 이용해 dataframe 만들기
        df = json_normalize(busan_stores, 'list')
        # dataframe에서 적절한 데이터만 가져와 출력하기(DT매장 dataframe 출력)
        df_dt = df[['s_name', 'lat', 'lot', 'sido_name', 'gugun_name', 'doro_address', 'tel']]
        df_dt = df_dt[df_dt['s_name'].str.contains('DT')].reset_index(drop = True)
        display(df_dt)
        # 지도 초기화면을 위한 매장들의 평균 위도, 경도 구하기
        # lat 값괴 lot의 데이터형식이 object이므로 float으로 바꾸어서 진행
        df_dt = df_dt.astype({'lat':'float'})
        df_dt = df_dt.astype({'lot':'float'})
        lat_mean = df_dt[['lat', 'lot']].describe(exclude = 'O').iloc[1,0]
        lot_mean = df_dt[['lat', 'lot']].describe(exclude = 'O').iloc[1,1]
        # 매장들의 위도, 경도 값을 받아 지도에 출력하기
        stores_lat = df_dt['lat']
        stores_lot = df_dt['lot']
        m = folium.Map(location=[lat_mean, lot_mean], zoom_start= 11)
        for mark_lat, mark_lot in zip(stores_lat, stores_lot):
            folium.Marker([mark_lat, mark_lot]).add_to(m)
        return m 
    except Exception as e:
        print(f'지역명을 확인하세요.\n입력한 지역 이름:{e}')

In [1]:
get_dt_location('전체')

NameError: name 'get_dt_location' is not defined

##### 지점간 거리를 구하는 코드 작성해보기

In [26]:
df_2 = df[['s_name', 'lat', 'lot', 'sido_name', 'gugun_name', 'doro_address', 'tel']]
df_2 = df_2.astype({'lat':'float'})
df_2 = df_2.astype({'lot':'float'})
dis_store_list = []
standard_list = []
other_list = []
dis_list = []
df_store_dis = pd.DataFrame()
i = 0
while True:
    standard_store = (df_2['lat'][i],df_2['lot'][i])
    for a, b in zip(df_2['lat'][i + 1:],df_2['lot'][i + 1:]):
        other_store = (a, b)
        stor_to_stor_dis = haversine(standard_store, other_store)
#         store_tup = (df_2['s_name'][i], df_2[df_2['lat'] == a]['s_name'], stor_to_stor_dis)
#         dis_store_list.append(store_tup)
        standard_list.append(df_2['s_name'][i])
        other_list.append(df_2[df_2['lat'] == a]['s_name'])
        dis_list.append(stor_to_stor_dis)
    if i == df_2.shape[0]-1:
        break
    i += 1
df_store_dis['기준 지점'] = standard_list
df_store_dis['연산할 다른 지점'] = other_list
df_store_dis['지점간 거리'] = dis_list
df_store_dis.sort_values('지점간 거리').head(10)


,기준 지점,연산할 다른 지점,지점간 거리
8555,제니스스퀘어,"119 마린시티아이파크 Name: s_name, dtype: object",0.061180
8366,센텀신세계B1,"128 센텀신세계3F Name: s_name, dtype: object",0.073758
8320,해운대,"131 그랜드조선 부산 Name: s_name, dtype: object",0.084636
2323,서면로R,"25 부산서면중앙로 Name: s_name, dtype: object",0.115844
3599,서면메디컬거리,"35 서면문화로 Name: s_name, dtype: object",0.122226
1860,부산역아스티,"16 부산역 Name: s_name, dtype: object",0.128663
8598,해운대이마트,"129 해운대중동역 Name: s_name, dtype: object",0.135953
3596,서면메디컬거리,"32 서면중앙 Name: s_name, dtype: object",0.136301
518,부산대역,"5 부산대 Name: s_name, dtype: object",0.138347
8641,센텀신세계3F,"130 센텀몰1F Name: s_name, dtype: object",0.143140


##### 지역명을 입력받아 지역 내 스타벅스 간의 거리를 계산하는 함수 만들어 보기

In [28]:
def get_stor_distance(location_name):
    '''
    지역명을 입력받아 지역 내 스타벅스 간의 거리를 계산하는 함수
    검색 가능 지역 : 전체, 서울, 경기, 광주, 대구, 대전, 부산, 울산,인천,
                     강원,경남, 경북, 전남, 전북, 충남, 충북, 제주, 세종
    '''
    try:
        # 검색 가능한 지역 목록
        location_dic = {
            '전체' : '00', '서울' : '01', '광주' : '02', '대구' : '03',
            '대전' : '04', '부산' : '05', '울산' : '06', '인천' : '07',
            '경기' : '08', '강원' : '09', '경남' : '10', '경북' : '11',
            '전남' : '12', '전북' : '13', '충남' : '14', '충북' : '15',
            '제주' : '16', '세종' : '17'
            }
        # 지역명을 입력받아 데이터를 불러오기 위한 번호 출력
        search_loc = location_dic[location_name]
        # 사이트에서 json으로 이루어진 정보 가져오기
        url = 'https://www.starbucks.co.kr/store/getStore.do?r=N22B447K1D'
        data = {
        'ins_lat' : '37.56682',
        'ins_lng' : '126.97865',
        'p_sido_cd' : search_loc,
        'in_biz_cd' : '',
        'set_date' : '',
        'iend' : '1000'}
        response = requests.post(url,data = data )
        location_stores = response.json()
        # json_normal을 이용해 dataframe 만들기
        df = json_normalize(location_stores, 'list')
        # dataframe에서 적절한 데이터만 가져와 출력하기(DT매장 dataframe 출력)
        df_2 = df[['s_name', 'lat', 'lot', 'sido_name', 'gugun_name', 'doro_address', 'tel']]
        df_2 = df_2.astype({'lat':'float'})
        df_2 = df_2.astype({'lot':'float'})
#         dis_store_list = []
        standard_list = []
        other_list = []
        dis_list = []
        df_store_dis = pd.DataFrame()
        i = 0
        while True:
            standard_store = (df_2['lat'][i],df_2['lot'][i])
            for a, b in zip(df_2['lat'][i + 1:],df_2['lot'][i + 1:]):
                other_store = (a, b)
                stor_to_stor_dis = haversine(standard_store, other_store)
        #         store_tup = (df_2['s_name'][i], df_2[df_2['lat'] == a]['s_name'], stor_to_stor_dis)
        #         dis_store_list.append(store_tup)
                standard_list.append(df_2['s_name'][i])
                other_list.append(df_2[df_2['lat'] == a]['s_name'])
                dis_list.append(stor_to_stor_dis)
            if i == df_2.shape[0]-1:
                break
            i += 1
        df_store_dis['기준 지점'] = standard_list
        df_store_dis['연산할 다른 지점'] = other_list
        df_store_dis['지점간 거리'] = dis_list
    except Exception as e:
        print('error : ', e)
    return df_store_dis.sort_values('지점간 거리').head(20)
#     return df_store_dis.sort_values('지점간 거리').head(10)

##### 함수가 잘 동작하는지 확인해보기

In [29]:
get_stor_distance('서울')

C:\Users\juku\AppData\Local\Temp\ipykernel_15880\1725392403.py:30: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(location_stores, 'list')


,기준 지점,연산할 다른 지점,지점간 거리
156100,신세계타임스퀘어B2,384 신세계타임스퀘어B2 389 신세계타임스퀘어1F Name: s_na...,0.000000
157574,타임스퀘어B2,"391 타임스퀘어B2 393 타임스퀘어2F Name: s_name, dt...",0.000000
162843,강북삼성병원,"418 강북삼성병원 451 강북삼성병원1층 Name: s_name, ...",0.000000
176957,현대디큐브2F,"535 현대디큐브B2 Name: s_name, dtype: object",0.031527
140054,잠실대교남단R,"333 잠실역 Name: s_name, dtype: object",0.033167
98923,합정폴리스R,"221 합정메세나몰 Name: s_name, dtype: object",0.041152
172787,무교동,"501 무교로 Name: s_name, dtype: object",0.048636
119091,센트럴F1,"274 센트럴시티 Name: s_name, dtype: object",0.049361
178881,서울역동자동,"589 포포인츠서울역 Name: s_name, dtype: object",0.054800
158395,여의도IFC(1F),"396 여의도신한투자증권타워 Name: s_name, dtype: object",0.056829


##### 지점간 거리가 0인 데이터를 없애는 코드 만들어 보기

In [30]:
region_list = ['서울', '경기', '광주', '대구', '대전', '부산', '울산','인천',
                     '강원','경남', '경북', '전남', '전북', '충남', '충북', '제주', '세종']
all_region_list = []
for region in region_list:
    all_region_list.append(get_stor_distance(region))
df_all = pd.concat(all_region_list)
df_result = df_all.sort_values('지점간 거리').head(100)
change_value = df_result['지점간 거리']*10**4
df_result['지점간 거리'] = change_value
df_result = df_result.astype({'지점간 거리':'string'})
df_result = df_result[~df_result['지점간 거리'].str.contains('0.0')]
df_result = df_result.astype({'지점간 거리':'float'})
recover_value = df_result['지점간 거리']/10**4
df_result['지점간 거리'] = recover_value
df_result.head(30)

C:\Users\juku\AppData\Local\Temp\ipykernel_15880\1725392403.py:30: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(location_stores, 'list')
C:\Users\juku\AppData\Local\Temp\ipykernel_15880\1725392403.py:30: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(location_stores, 'list')
C:\Users\juku\AppData\Local\Temp\ipykernel_15880\1725392403.py:30: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(location_stores, 'list')
C:\Users\juku\AppData\Local\Temp\ipykernel_15880\1725392403.py:30: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(location_stores, 'list')
C:\Users\juku\AppData\Local\Temp\ipykernel_15880\1725392403.py:30: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  

,기준 지점,연산할 다른 지점,지점간 거리
1541,신세계대구3F(티바나),"24 신세계대구8FR Name: s_name, dtype: object",0.006440
79895,스타필드고양3F(티바나),"291 스타필드고양1FR Name: s_name, dtype: object",0.027764
176957,현대디큐브2F,"535 현대디큐브B2 Name: s_name, dtype: object",0.031527
140054,잠실대교남단R,"333 잠실역 Name: s_name, dtype: object",0.033167
98923,합정폴리스R,"221 합정메세나몰 Name: s_name, dtype: object",0.041152
172787,무교동,"501 무교로 Name: s_name, dtype: object",0.048636
119091,센트럴F1,"274 센트럴시티 Name: s_name, dtype: object",0.049361
178881,서울역동자동,"589 포포인츠서울역 Name: s_name, dtype: object",0.054800
158395,여의도IFC(1F),"396 여의도신한투자증권타워 Name: s_name, dtype: object",0.056829
151391,타임스퀘어R,384 신세계타임스퀘어B2 389 신세계타임스퀘어1F Name: s_na...,0.058503
